In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import numpy as np
import glob
import time

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : os.getcwd()}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
service = ChromeService(executable_path=ChromeDriverManager().install())

C:\Users\ebroh\AppData\Local\Temp\ipykernel_88784\1264243978.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\ebroh\AppData\Local\Temp\ipykernel_88784\1264243978.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [3]:
def openBudgetSite(driver, school_code, fiscal_year, budgetOrAllocation = 'budget'):
    if (budgetOrAllocation == 'allocation'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxyallocation/default.aspx")

    elif (budgetOrAllocation == 'budget'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxybudgetsummaryto/default.aspx")
        
    else:
        raise ValueError('Budget or Allocation value must be either "budget" or "allocation"')
                         
    # get element 
    element = driver.find_element(By.XPATH,'//*[@id="School_Code"]')

    # send keys 
    element.send_keys(school_code)

    x = driver.find_element(By.XPATH,'//*[@id="Fiscal_Year"]')
    drop=Select(x)

    # select by visible text
    drop.select_by_visible_text(fiscal_year)

    driver.find_element(By.XPATH,'//*[@id="Enter"]').click()


    try:
        element = driver.find_element(By.XPATH,'//*[@id="messag"]/div[1]/div[3]/h2/a')
        element.text.index(location_codes[5])
        return True

    except Exception as e:
        if hasattr(e, 'message'):
            return e.message
        else:
            return e

def allocationPageScraper(driver, school_code, fiscal_year):
    list_of_dfs = pd.read_html(driver.page_source)
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'allocation_category', 'amount'])

    for df in list_of_dfs:
        df = df.rename(columns={df.columns[0]: 'allocation_category', df.columns[1]: 'amount'})
        df = df[df["allocation_category"].str.contains("total", case = False)==False]
        df['location_code'] = school_code
        df['fiscal_year'] = fiscal_year
        final_df = pd.concat([final_df, df])
    
    return final_df
    
    
def budgetPageScraper(driver, school_code, fiscal_year):
    section_titles = driver.find_elements(By.CLASS_NAME , 'TO_Section')
    list_of_dfs = pd.read_html(driver.page_source)
    
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'budget_category', 'budget_assignment', 'num_positions', 'service_type', 'amount'])

    for i,df in enumerate(list_of_dfs):
        if (len(df.columns) == 2):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['num_positions'] = None
            df['service_type'] = None
            final_df = pd.concat([final_df, df])
        
        elif (len(df.columns) == 3):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'num_positions', df.columns[2]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['service_type'] = None
            final_df = pd.concat([final_df, df])

        elif (len(df.columns) == 4):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'service_type', df.columns[2]: 'num_positions', df.columns[3]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df = df[df["service_type"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            final_df = pd.concat([final_df, df])
            
    return final_df

In [4]:
if (openBudgetSite(driver, 'M125', '2022', 'budget')):
    df = budgetPageScraper(driver, 'M125', '2022')

In [5]:
df

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount
0,M125,2022,Leadership,HEAD OF SCHOOL,1.0,None,"$ 84,471"
1,M125,2022,Leadership,AP - SUPERVISION,1.0,None,"$ 121,384"
0,M125,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,None,"$ 53,241"
0,M125,2022,Secretary,School Secretary,1.0,None,"$ 76,956"
0,M125,2022,Classroom Teacher,Pre K,2.0,WHOLE CLASS,"$ 178,549"
1,M125,2022,Classroom Teacher,K,2.0,WHOLE CLASS,"$ 207,632"
2,M125,2022,Classroom Teacher,"K,1",1.0,SELF-CONTAINED SP ED,"$ 97,194"
3,M125,2022,Classroom Teacher,1,2.0,WHOLE CLASS,"$ 198,553"
4,M125,2022,Classroom Teacher,2,2.0,WHOLE CLASS,"$ 194,824"
5,M125,2022,Classroom Teacher,3,2.0,WHOLE CLASS,"$ 231,099"


In [6]:
if(openBudgetSite(driver, 'M125', '2022', 'allocation')):
    df = allocationPageScraper(driver, 'M125', '2022')

In [7]:
df

,location_code,fiscal_year,allocation_category,amount
0,M125,2022,4K for All,"$ 475,301"
1,M125,2022,ARPA Academic Recovery,"$ 79,229"
2,M125,2022,ARPA Academic Recovery AP Summer Rising,"$ 17,990"
3,M125,2022,ARPA Academic Recovery Arts,"$ 19,807"
4,M125,2022,ARPA Academic Recovery SE After-School Instruc...,"$ 24,609"
...,...,...,...,...
7,M125,2022,TL RS Speech Sixth Period Coverage,"$ 6,143"
8,M125,2022,TL SBST Shared,"$ 17,138"
9,M125,2022,TL Situation Room,"$ 3,953"
10,M125,2022,TL Terminal and Paid Leaves,"$ 13,502"
